In [15]:
from datasets import load_dataset
from adapters import AutoAdapterModel

from transformers import BertConfig
from transformers import BertTokenizer
import wandb

In [16]:
wandb.login(key="d1209ac0e4a568567e407d3afef99099ea87a4b1")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [17]:
%env WANDB_PROJECT=peft
%env WANDB_ENTITY=SC4001

env: WANDB_PROJECT=peft
env: WANDB_ENTITY=SC4001


In [18]:
dataset = load_dataset("rotten_tomatoes")
dataset.num_rows

Path, databilder:  rotten_tomatoes


{'train': 8530, 'validation': 1066, 'test': 1066}

In [19]:

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def encode_batch(batch):
    """Encodes a batch of input data using the model tokenizer."""
    return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

dataset = dataset.map(encode_batch, batched=True)
dataset = dataset.rename_column(original_column_name="label", new_column_name="labels")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/8530 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [52]:
from adapters import IA3Config, SeqBnConfig, LoRAConfig, ReftConfig, CompacterConfig

ia3_config = IA3Config()
seq_bn_config = SeqBnConfig()
lora_config = LoRAConfig()
compacter_config = CompacterConfig()
reft_config = ReftConfig(layers="all", prefix_positions=3, suffix_positions=0, r=1, orthogonality=True)


In [78]:
config = BertConfig.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
)
model = AutoAdapterModel.from_pretrained(
    "bert-base-uncased",
    config=config,
)

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [83]:
runname = "bert-rotten_tomatoes-lora"
model.add_adapter("rotten_tomatoes", config=lora_config)
model.add_classification_head(
    "rotten_tomatoes",
    num_labels=2,
    id2label={ 0: "👎", 1: "👍"}
)

# Activate the adapter
model.train_adapter("rotten_tomatoes")

# calculate the number of training parameters
total = 0
for p in model.parameters():
    if p.requires_grad:
        total += p.numel()
print(f"Total number of trainable parameters: {total}")

ValueError: An adapter with the name 'rotten_tomatoes' has already been added.

In [ ]:
import numpy as np
from transformers import TrainingArguments, EvalPrediction
from adapters import AdapterTrainer

training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=6,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    run_name=runname
)

def compute_accuracy(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [84]:
trainer.train()

/datadrive5/huypn16/anaconda3/envs/ana/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
200,0.406600


wandb: ERROR Error while calling W&B API: Error 3988 (HY000): Conversion from collation utf8mb4_0900_ai_ci into utf8mb3_general_ci impossible for parameter (<Response [422]>)


TrainOutput(global_step=204, training_loss=0.407117425811057, metrics={'train_runtime': 306.3164, 'train_samples_per_second': 167.082, 'train_steps_per_second': 0.666, 'total_flos': 2135229632755200.0, 'train_loss': 0.407117425811057, 'epoch': 6.0})

In [85]:
trainer.evaluate(dataset["validation"])

/datadrive5/huypn16/anaconda3/envs/ana/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.37011364102363586,
 'eval_acc': 0.8311444652908068,
 'eval_runtime': 3.1652,
 'eval_samples_per_second': 336.785,
 'eval_steps_per_second': 1.58,
 'epoch': 6.0}

wandb: ERROR Error while calling W&B API: Error 3988 (HY000): Conversion from collation utf8mb4_0900_ai_ci into utf8mb3_general_ci impossible for parameter (<Response [422]>)
